### Imports

In [1]:
import numpy as np
np.set_printoptions(suppress=True)
import matplotlib.pyplot as plt
import cvxpy as cp 

Question  d)

In [2]:
from scipy.optimize import minimize, NonlinearConstraint, Bounds, LinearConstraint
from numpy import cos, pi

In [3]:
# constants
n = 10
alpha = 1.5
rmin = 1.0
rmax = 2.0
theta = 2 * pi / (5 * (n + 1))
costh = cos(theta)  # radians

In [4]:
### Linear constraints ###
lin_const = []

# rmin <= r_i <= rmax (2)
A2 = LinearConstraint(np.eye(n), rmin, rmax)
lin_const.append(A2)

# -theta*alpha <= r_{i+1} - r_i <= theta*alpha for i = 1, ..., n (4)
M = (np.eye(n+1) - np.eye(n+1, k=-1))[1:-1, :-1]

# col 1 is r1, col 2 is r2, etc.
A4 = LinearConstraint(M, -theta*alpha, theta*alpha)
lin_const.append(A4)

# i = 0
Mr1 = np.append(1, np.zeros(n-1, dtype=int))
Ar1 = LinearConstraint(Mr1, rmin-theta*alpha, rmin+theta*alpha)
lin_const.append(Ar1)

# i = n
Mrn = np.append(np.zeros(n-1, dtype=int), 1)
Arn = LinearConstraint(Mrn, rmax-theta*alpha, rmax+theta*alpha)
lin_const.append(Arn)

In [5]:
### Nonlinear constraints ###
# 2r_{i-1}*r_{i+1}*cos(theta) - r_i*r_{i-1} - r_i*r_{i+1} <= 0 for i = 0, ..., n+1 (3)
non_lin_const = []

# i = 0
nlcon_0 = NonlinearConstraint(lambda r: 2*rmin*r[0]*costh - rmin*rmin - rmin*r[0], -np.inf, 0)
non_lin_const.append(nlcon_0)

# i = 1
nlcon_1 = NonlinearConstraint(lambda r: 2*rmin*r[1]*costh - r[0]*rmin - r[0]*r[1], -np.inf, 0)
non_lin_const.append(nlcon_1)

# i = 2...n-1
# for some reason a for loop that creates lambda functions doesn't work, messes up indexes?!?
non_lin_const.append(NonlinearConstraint(lambda r: 2*r[0]*r[2]*costh - r[1]*r[0] - r[1]*r[2], -np.inf, 0))
non_lin_const.append(NonlinearConstraint(lambda r: 2*r[1]*r[3]*costh - r[2]*r[1] - r[2]*r[3], -np.inf, 0))
non_lin_const.append(NonlinearConstraint(lambda r: 2*r[2]*r[4]*costh - r[3]*r[2] - r[3]*r[4], -np.inf, 0))
non_lin_const.append(NonlinearConstraint(lambda r: 2*r[3]*r[5]*costh - r[4]*r[3] - r[4]*r[5], -np.inf, 0))
non_lin_const.append(NonlinearConstraint(lambda r: 2*r[4]*r[6]*costh - r[5]*r[4] - r[5]*r[6], -np.inf, 0))
non_lin_const.append(NonlinearConstraint(lambda r: 2*r[5]*r[7]*costh - r[6]*r[5] - r[6]*r[7], -np.inf, 0))
non_lin_const.append(NonlinearConstraint(lambda r: 2*r[6]*r[8]*costh - r[7]*r[6] - r[7]*r[8], -np.inf, 0))
non_lin_const.append(NonlinearConstraint(lambda r: 2*r[7]*r[9]*costh - r[8]*r[7] - r[8]*r[9], -np.inf, 0))

# i = n
nlcon_n = NonlinearConstraint(lambda r: 2*r[n-2]*rmax*costh - r[n-1]*r[n-2] - r[n-1]*rmax, -np.inf, 0)
non_lin_const.append(nlcon_n)

# i = n+1
nlcon_np1 = NonlinearConstraint(lambda r: 2*r[n-1]*r[n-1]*costh - rmax*r[n-1] - rmax*r[n-1], -np.inf, 0)
non_lin_const.append(nlcon_np1)

In [6]:
print(len(non_lin_const))

12


In [7]:
### Objective function ###
# maximize sum of r_i
def obj(r):
    return -np.mean(r)

In [8]:
# initial design (r0)
r0 = rmin*np.ones(n)

### Solve ###
res = minimize(obj, r0,
    method="trust-constr",
    constraints=[*lin_const, *non_lin_const],
    options={"maxiter": 10000}
)

# print results
print(res)

 barrier_parameter: 3.200000000000001e-05
 barrier_tolerance: 3.200000000000001e-05
          cg_niter: 35
      cg_stop_cond: 4
            constr: [array([1.01320359, 1.04049947, 1.08381755, 1.14648066, 1.23388601,
       1.3548074 , 1.52398202, 1.69521275, 1.86610606, 1.99984003]), array([0.02729588, 0.04331807, 0.06266312, 0.08740535, 0.12092138,
       0.16917462, 0.17123073, 0.17089332, 0.13373397]), array([1.01320359]), array([1.99984003]), array([-0.00000507]), array([-0.00000696]), array([-0.0000095]), array([-0.00001383]), array([-0.00002154]), array([-0.00003251]), array([-0.00005397]), array([-0.08474314]), array([-0.09617111]), array([-0.15924761]), array([-0.31582451]), array([-0.0527776])]
       constr_nfev: [0, 0, 0, 0, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286, 286]
       constr_nhev: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
       constr_njev: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    constr_penalty: 1.0
  constr_violation: 0.0
    exe

c:\Users\lange\.conda\envs\4dm20\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


In [9]:
# print results
print(np.round(res.x,4))

[1.0132 1.0405 1.0838 1.1465 1.2339 1.3548 1.524  1.6952 1.8661 1.9998]


`[1.0132 1.0405 1.0839 1.1466 1.2341 1.3551 1.5246 1.6959 1.8673 2.0000]`